Для первой задачи мы используем данные [Jester Online Joke Recommender System](https://goldberg.berkeley.edu/jester-data/)

**Описание данных**

Файл `train_joke_df.csv` содержит:
- UID - id пользователей
- JID - id шуток, которые 
- Ratin - рейтинг шутки, который проставил пользователь 


Рейтинг имеет значение от -10.00 до 10.00. Могут встречаться значения 99.00, но это обозначает Null (нет рейтинга от пользователя).

Метрика для оценки [RMSE](https://www.codecamp.ru/blog/how-to-interpret-rmse/)

Минимальный RMSE: `4.2238`



In [1]:
#!pip install "scikit-surprise==1.1.3"

In [2]:
#!pip install "xlrd==2.0.1"

### Import

In [3]:
import numpy as np
import pandas as pd
from collections import defaultdict
from surprise import Dataset, Reader, KNNWithMeans, accuracy
from surprise.model_selection import GridSearchCV
from surprise.model_selection import train_test_split
from sklearn.model_selection import train_test_split as tts
from surprise.model_selection import KFold
from tqdm.notebook import tqdm

np.random.seed(42)

In [ ]:
n_clust = 20000

print(n_clust)

### Базовые функции для скоринга и получения рекомендаций

In [4]:
def get_num_user_ratings(uid):
    """ возвращает кол-во рейтингов у пользователя 
    args: 
      uid: id пользователей
    returns: 
      кол-во объектов, которые оценил пользователь
    """
    try:
        return len(trainset.ur[trainset.to_inner_uid(uid)])
    except ValueError: # пользователя не было во время обучения (новый, отправить на стартовые рекомендации)
        return 0
    
def get_num_item_ratings(iid):
    """ возвращает кол-во пользователей, которые оценили выбранный элемент 
    args:
      iid: строка с элементов рекомендации
    returns:
      кол-во пользователей, которые дали оценки по элементу
    """
    try: 
        return len(trainset.ir[trainset.to_inner_iid(iid)])
    except ValueError:
        return 0
    
# На основе Surprise FAQ построим рекомендации Топ-N
def get_top_n(predictions, n=5):
    """Определят Топ-N рекомендаций

    Args:
        predictions(list of Prediction objects): Списко рекомендаций, из алгоритма Surprise
        n(int): Кол-во топ рекомендаций

    Returns:
        Словарь пользователь - список рекомендакиций для пользователей
        [(raw item id, rating estimation), ...]
    """

    # Предикт для каждого пользователя
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Сортировка предикта (по пользователям)
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n    

### Загрузка и обработка данных

In [5]:
df = pd.read_csv(r'..\data\recsys-in-practice\train_joke_df.csv')

df.head(5)

,UID,JID,Rating
0,18029,6,-1.26
1,3298,64,-4.17
2,3366,58,0.92
3,12735,92,3.69
4,11365,38,-6.60


In [6]:
cluster_df = pd.read_csv(f'cluster_df_{n_clust}.csv', index_col=1)
cluster_df

,cluster_id
UID,
1,3314
2,389
3,5978
4,16224
5,6296
...,...
24979,824
24980,1003
24981,19710


In [7]:
df = df.merge(cluster_df, left_on='UID', right_index=True)
df

,UID,JID,Rating,cluster_id
0,18029,6,-1.26,16091
18990,18029,40,8.30,16091
50486,18029,7,3.30,16091
69643,18029,16,-9.27,16091
124589,18029,15,8.11,16091
...,...,...,...,...
1303561,21379,62,0.15,8300
1331118,21379,46,1.31,8300
1368576,21379,49,-0.68,8300
1376073,21379,54,2.33,8300


In [8]:
df.head(5)

,UID,JID,Rating,cluster_id
0,18029,6,-1.26,16091
18990,18029,40,8.30,16091
50486,18029,7,3.30,16091
69643,18029,16,-9.27,16091
124589,18029,15,8.11,16091


In [9]:
# сделаем сортировку и перепишем index
df = df.sort_values(by=['cluster_id', 'JID'])
df = df.reset_index(drop=True)

In [10]:
# создадим на основе набора данных
# поднабор, который требуется для библиотеки Surprise

# указываем минимальный и максимальный рейтинги
reader = Reader(rating_scale=(-10, 10))

# передаём набор, указывая последовательность колонок: user (raw) ids, item (raw) ids, ratings
# для Surprise - это обязательно
data = Dataset.load_from_df(df[['cluster_id', 'JID', 'Rating']], reader)

In [11]:
trainset_data = data.build_full_trainset()

# сделаем разделение на обучающую и тестовую выборку
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

### Обучение модели

In [12]:
# определим набор данных для GridSearchCV
sim_options = {
    "name": ["msd", "cosine"], # способы оценки похожести (в GridSearch)
    "min_support": [3, 4],     # минимальное кол-во общих пользоватлей с данной шуткой
    "user_based": [False],     # поиск "похожести" будет на основе шуток, а не пользователей
}

param_grid = {"sim_options": sim_options}

gs = GridSearchCV(KNNWithMeans, param_grid, measures=["rmse", "mae"], cv=2,)
gs.fit(data)
     
# результат
print(gs.best_score["rmse"])
print(gs.best_params["rmse"])

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
4.26333994470934
{'sim_options': {'name': 'cosine', 'min_support': 3, 'user_based': False}}


In [13]:
# обучим с лучшими параметрами
algo = gs.best_estimator['rmse']
algo.fit(trainset)

# получим предикт и посмотрим метрику
predictions = algo.test(testset)
accuracy.rmse(predictions)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 4.2152


4.215185605584823

### Тестирование и результаты

In [14]:
# получаем предикт
uid = 1  # id пользователя 
iid = 1  # iв шутки

# получим предик на основе обученных данных
# -7.82 - это фактический рейтинг, но посмотрим, какой ответ будет в предикте
pred = algo.predict(uid, iid, r_ui=-7.82, verbose=True)

user: 1          item: 1          r_ui = -7.82   est = 2.83   {'actual_k': 28, 'was_impossible': False}


In [15]:
uid = 24983  # id пользователя 
iid = 62     # iв шутки

pred = algo.predict(uid, iid, r_ui=-0.29, verbose=True)

user: 24983      item: 62         r_ui = -0.29   est = 0.88   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}


### Обзор рекомендаций

In [16]:
# построим таблицу для обзора набора рекомендаций
# посмотрим, какие элементы и в каком кол-ве рекомендуем
trainset = algo.trainset

predictions_df = pd.DataFrame(predictions, columns=['uid', 'iid', 'rui', 'est', 'details'])

predictions_df['№ кол-во пользовательских рейтингов'] = predictions_df.uid.apply(get_num_user_ratings)
predictions_df['№ кол-во рейтингов элементов'] = predictions_df.iid.apply(get_num_item_ratings)
predictions_df['error'] = abs(predictions_df.est - predictions_df.rui)

best_predictions = predictions_df.sort_values(by='error')[:10]
worst_predictions = predictions_df.sort_values(by='error')[-10:]

In [17]:
best_predictions.head(5)

,uid,iid,rui,est,details,№ кол-во пользовательских рейтингов,№ кол-во рейтингов элементов,error
85181,13658,48,-0.29,-0.290024,"{'actual_k': 39, 'was_impossible': False}",44,15774,0.000024
135465,12304,28,3.06,3.059971,"{'actual_k': 40, 'was_impossible': False}",70,15366,0.000029
208376,297,36,4.32,4.320046,"{'actual_k': 24, 'was_impossible': False}",28,15818,0.000046
29359,898,68,2.28,2.279898,"{'actual_k': 40, 'was_impossible': False}",816,15890,0.000102
96873,2676,42,1.26,1.260103,"{'actual_k': 40, 'was_impossible': False}",591,15798,0.000103


In [18]:
# Предикт для всех, кого нет в выборке для обучения
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n = get_top_n(predictions)

# Сделаем вывод рекомендаций
a=0
for uid, user_ratings in top_n.items():
    a+=1
    print(uid, [iid for (iid, _) in user_ratings])
    
    if a==10:
        break

14078 [50, 89, 36, 32, 35]
5382 [27, 32, 35, 53, 69]
3609 [81, 88, 91, 98, 97]
1869 [89, 72, 83, 76, 81]
2016 [53, 54, 76, 69, 93]
6398 [72, 93, 50, 91, 62]
3413 [62, 93, 83, 78, 91]
18259 [89, 29, 35, 72, 69]
1761 [89, 93, 78, 97, 84]
12526 [89, 36, 32, 72, 54]


### Для отправки на тестирование

In [19]:
test = pd.read_csv(r'..\data\recsys-in-practice\test_joke_df_nofactrating.csv', index_col=0)
test.head(5)

,UID,JID
InteractionID,,
0,11228,39
1,21724,85
2,16782,56
3,12105,42
4,14427,2


In [20]:
test = test.join(cluster_df, on='UID', lsuffix='_left', rsuffix='_right')
test

,UID,JID,cluster_id
InteractionID,,,
0,11228,39,5294
1,21724,85,1760
2,16782,56,5080
3,12105,42,11472
4,14427,2,7126
...,...,...,...
362086,3085,66,2450
362087,13765,31,4606
362088,10341,29,2767


In [21]:
test['Rating'] = test[['cluster_id', 'JID']].apply(lambda x: algo.predict(x[0], x[1], verbose=False).est,
                                                      axis = 1)
                                                      


In [22]:
# вид набора данных, который должен быть отправлен для тестирования
test['Rating'].to_frame().head(5)

,Rating
InteractionID,
0,0.843270
1,-5.392343
2,-1.536955
3,7.508531
4,6.885926


In [23]:
# формирование файла для отправки в Kaggle
test['Rating'].to_frame().to_csv(f'baseline_cluster_{n_clust}.csv')

In [24]:
algo.predict(11228, 39, verbose=False)

Prediction(uid=11228, iid=39, r_ui=None, est=-3.1336757530134327, details={'actual_k': 40, 'was_impossible': False})

In [25]:
algo.estimate(u=11228, i=39)

(0.07572168237170773, {'actual_k': 40})

In [26]:
algo.sim.shape

(100, 100)